In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Load your data
df = pd.read_csv('hypertensions.csv')

# Convert 'Sold_date' to datetime format
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')

# Set 'Sold_date' as the index
df.set_index('Sold_date', inplace=True)

# Aggregate sales data on a weekly basis for each product
weekly_sales = df.groupby([pd.Grouper(freq='W'), 
'Product_details']).agg(total_sold=('Sold_quantity', 'sum')).reset_index()

# Apply one-hot encoding to 'Product_details'
weekly_sales_encoded = pd.concat([weekly_sales, pd.get_dummies(weekly_sales['Product_details'], 
prefix='product')], axis=1).drop('Product_details', axis=1)

# Extract year and week number from 'Sold_date' for temporal features
weekly_sales_encoded['year'] = weekly_sales_encoded['Sold_date'].dt.year
weekly_sales_encoded['week_of_year'] = weekly_sales_encoded['Sold_date'].dt.isocalendar().week

# Normalize the temporal features 
scaler = MinMaxScaler()
features_columns = ['year', 'week_of_year'] + [col for col in weekly_sales_encoded.columns 
if col.startswith('product')]
weekly_sales_encoded[features_columns] = scaler.fit_transform(weekly_sales_encoded[features_columns])

# Split the data into features and target
X = weekly_sales_encoded[features_columns].values
y = weekly_sales_encoded['total_sold'].values

# Adjusting the dataset split into 70% training, 15% validation, and 15% testing
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  # 0.1765 of 85% is roughly 15% of the whole

# Define the model architecture
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(1, activation='relu')    # Output layer
])


# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model with the validation data
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=10)

# Evaluate the model
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE on test set: {rmse}')

# Prepare forecasting features for Feb 1 to 8, 2024, correctly this time
forecast_results = []
for product_col in [col for col in features_columns if col.startswith('product')]:
    forecast_period_features = np.zeros((1, len(features_columns)))
    # Find the index of the year, week_of_year, and current product column
    year_index = features_columns.index('year')
    week_of_year_index = features_columns.index('week_of_year')
    product_index = features_columns.index(product_col)
    
    # Normalize year and week_of_year values
    forecast_period_features[0, year_index] = (2024 - scaler.data_min_[year_index]) / (scaler.data_max_[year_index] - scaler.data_min_[year_index])
    forecast_period_features[0, week_of_year_index] = (5 - scaler.data_min_[week_of_year_index]) / (scaler.data_max_[week_of_year_index] - scaler.data_min_[week_of_year_index])
    forecast_period_features[0, product_index] = 1  # Activate current product

    # Predict and store the result
    predicted_quantity = model.predict(forecast_period_features).flatten()[0]
    product_name = product_col.replace('product_', '')  # Assuming prefix 'product_' is used
    forecast_results.append((product_name, predicted_quantity))

# Display forecasted quantities for each product in a DataFrame
forecast_df = pd.DataFrame(forecast_results, columns=['Product', 'Forecasted Quantity']).sort_values(by='Forecasted Quantity', ascending=False)
print(forecast_df)

Epoch 1/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7871.2466 - val_loss: 4050.8936
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - loss: 3696.9246 - val_loss: 2896.5439
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step - loss: 2324.2634 - val_loss: 2470.9988
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - loss: 1929.7715 - val_loss: 2409.0774
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - loss: 2069.6860 - val_loss: 2395.9541
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step - loss: 2057.6084 - val_loss: 2355.0215
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - loss: 2375.2725 - val_loss: 2434.9978
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step - loss: 2252.2334 - val_loss: 2349.3721
Epoch 9/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - loss: 2326.2991 - val_loss: 2340.7097
Epoch 10/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step - loss: 2228.3293 - val_loss: 2350.7905
Epoch 11/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step - loss: 2159.7983

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - loss: 1907.7395 - val_loss: 2185.1931
Epoch 66/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step - loss: 2026.2960 - val_loss: 2080.4385
Epoch 67/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - loss: 1823.3506 - val_loss: 2052.9561
Epoch 68/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step - loss: 2204.7803 - val_loss: 2055.1289
Epoch 69/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step - loss: 1735.4962 - val_loss: 2096.3418
Epoch 70/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step - loss: 1830.9249 - val_loss: 2065.0781
Epoch 71/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - loss: 1816.9725 - val_loss: 2124.9438
Epoch 72/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step - loss: 1874.9136 - val_loss: 2046.5760
Epoch 73/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 477us/step - loss: 1629.7289 - val_loss: 2087.3550
Epoch 74/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - loss: 1441.3625 - val_loss: 2063.3672
Epoch 75/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step - loss: 2140.1421 -